# Computer Vision: YOLO for identifying Formula 1 Teams


In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO
import cv2

In [ ]:

# Ver versión de PyTorch
print(f"Versión de PyTorch: {torch.__version__}")

# Ver disponibilidad de CUDA
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA no está disponible.")

Versión de PyTorch: 2.5.1+cu124
CUDA disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
# Configuración del dispositivo (GPU o CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parámetros del modelo y entrenamiento
NUM_CLASSES = 10  # Número de equipos de F1
LEARNING_RATE = 0.0005
BATCH_SIZE = 8  # Ajusta según la memoria de la GPU
FINE_TUNED_BATCH_SIZE = 6
NUM_WORKERS = 8  # Aumentar si tienes más núcleos en la CPU

NUM_EPOCHS = 50  # O más, dependiendo del rendimiento
FINE_TUNED_EPOCHS = 30
AUGMENTATION = True  # Hacer augmentación de datos

In [ ]:
import cv2
import numpy as np

# Verificar si las imágenes se leen correctamente
train_image = cv2.imread(
    "../f1-strategy/f1-dataset/train/images/1-2023-Brazilian-GP-FP1-20_jpg.rf.2f3d25867c8e1661f6070ae5a84c6dd4.jpg")
if train_image is None:
    print("Error al leer la imagen")
else:
    print("Imagen cargada correctamente: ", train_image.shape)

Imagen cargada correctamente:  (640, 640, 3)


In [ ]:
import cv2
import os

# Definir la carpeta de imágenes
image_folder = "C:/Users/victo/Desktop/Documents/Tercer año/Segundo Cuatrimestre/Finales/f1-strategy/f1-dataset/train/images"

# Lista para almacenar los tamaños de las imágenes
sizes = set()

# Verificar el tamaño de cada imagen
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    img = cv2.imread(image_path)

    if img is not None:
        img_size = img.shape  # (height, width, channels)
        sizes.add(img_size)
    else:
        print(f"Error al cargar la imagen: {image_name}")

# Comprobar si todas las imágenes tienen el mismo tamaño
if len(sizes) == 1:
    print("Todas las imágenes tienen el mismo tamaño.")
else:
    print("Hay imágenes con tamaños diferentes. Los tamaños encontrados son:")
    for size in sizes:
        print(size)

Todas las imágenes tienen el mismo tamaño.


In [ ]:
# Cargar el modelo YOLOv8 Nano
from torch.optim import Adam
model = YOLO("yolov8m.pt")


# model.train(
#     data="../f1-strategy/f1-dataset/data.yaml",
#     epochs=NUM_EPOCHS,
#     batch=BATCH_SIZE,
#     optimizer="SGD",         # Puedes probar también con "AdamW" para un optimizador adaptativo
#     lr0=LEARNING_RATE,
#     momentum=0.9,
#     warmup_epochs=5,         # Menos epochs de warmup para entrar más rápido en el entrenamiento efectivo
#     weight_decay=0.0001,
#     label_smoothing=0.1,
#     degrees=15.0,            # Rotación máxima reducida a 15° para evitar transformaciones demasiado agresivas
#     mixup=0.15,              # Mezcla de imágenes de forma moderada
#     copy_paste=0.05,         # Menor intensidad para no alterar en exceso la composición real de la imagen
#     hsv_h=0.015,             # Cambios sutiles en el tono para mantener la naturalidad
#     box=7.5,
#     cls=1.5,
#     patience=10,             # Más paciencia para que el scheduler pueda adaptarse y reducir la lr si es necesario
#     pretrained=True,
#     project='../f1-strategy/yolo-files/runs',
#     name='train_v1',
# )


# SOBREAJUSTE Y BAJO RECALL, NUEVO MODELO
model.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=NUM_EPOCHS,
    batch=BATCH_SIZE,
    optimizer="SGD",
    lr0=LEARNING_RATE,
    lrf=0.01,               # Factor LR final para scheduler cosine
    cos_lr=True,            # Activar cosine annealing scheduler
    momentum=0.9,
    warmup_epochs=3,        # Warmup más corto
    weight_decay=0.0005,    # Regularización aumentada

    # Balanceo de clases y prevención de sobreajuste
    box=7.5,                # Mayor peso en pérdida de detección de cajas
    cls=2.5,                # Mayor peso para la pérdida de clasificación

    # Data augmentation intensificado para mejorar generalización
    mosaic=1.0,             # Máximo mosaico para objetos pequeños/distantes
    mixup=0.2,              # Mezcla de imágenes moderada
    copy_paste=0.1,         # Copiar-pegar para aumentar variabilidad
    degrees=20.0,           # Más rotación para variabilidad
    translate=0.2,          # Mayor traslación
    scale=0.2,              # Escala para simular objetos de diferentes tamaños
    shear=0.1,              # Distorsión ligera
    perspective=0.0001,     # Ligero cambio de perspectiva
    fliplr=0.5,             # Flip horizontal con 50% de probabilidad
    flipud=0.05,            # Flip vertical ocasional
    hsv_h=0.015,            # Cambios sutiles en colores
    hsv_s=0.2,              # Mayor saturación para distinguir colores
    hsv_v=0.1,              # Variación en brillo

    # Parámetros de entrenamiento
    patience=15,            # Más paciencia para el early stopping
    verbose=True,
    device=DEVICE,
    pretrained=True,        # Usar pesos preentrenados
    project='../f1-strategy/yolo-files/runs',
    name='train_optimized',

    # Guardar mejores modelos
    save_period=5,          # Guardar cada 5 épocas
)

New https://pypi.org/project/ultralytics/8.3.79 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=50, time=None, patience=15, batch=8, imgsz=640, save=True, save_period=5, cache=False, device=cuda, workers=8, project=../f1-strategy/yolo-files/runs, name=train_optimized3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_optimized3\labels.jpg... 
optimizer: SGD(lr=0.0005, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_optimized3
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         29      0.583      0.837      0.794      0.646



50 epochs completed in 0.933 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_optimized3\weights\last.pt, 52.0MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_optimized3\weights\best.pt, 52.0MB

Validating ..\f1-strategy\yolo-files\runs\train_optimized3\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.71it/s]


                   all         20         29      0.653      0.795      0.815      0.665
           Kick Sauber          4          6      0.794        0.5       0.71      0.508
          Racing Bulls          1          1      0.467          1      0.995      0.796
                Alpine          4          5      0.797      0.787      0.811      0.674
               Ferarri          3          4      0.625       0.75      0.788      0.705
                  Haas          1          1      0.474          1      0.995      0.895
               Mclaren          5          6      0.807        0.5      0.775      0.651
              Mercedes          3          3          1      0.826      0.995      0.735
              Williams          2          3      0.259          1       0.45      0.355
Speed: 0.3ms preprocess, 8.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_optimized3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021028FEEBF0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

# Resultados con 200 imágenes

### Model With this paremeters and no optimizer:

#### Configuración del dispositivo (GPU o CPU)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Parámetros del modelo y entrenamiento

NUM_CLASSES = 10 # Número de equipos de F1
BATCH_SIZE = 8
NUM_WORKERS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 0.005

Mal rendimiento, pasando a nuevos valores y reentrenamiento.

### Aun mal rendimiento, Pasando a un modelo mas complejo, del nano al m.

### Nuevo modelo tras data augmentation

Después de aumentar las imágenes a 200 por clase, rendimiento del 70% obtenido en todas las clases.
Sí, recuerdo los resultados anteriores que compartiste. Por ejemplo, tenías:

Métricas generales:

mAP50: 0.772
mAP50-95: 0.449
Precisión (P): 0.733
Recall (R): 0.615
Clases problemáticas y sus mAP50:

Williams: 0.487
Kick Sauber: 0.619 (con un recall notablemente bajo, alrededor de 0.443)
Alpine: 0.556
Clases con buenos resultados:

Racing Bulls y Haas: mAP50 > 0.99
Mclaren y Mercedes: mAP50 > 0.88
Tiempo de inferencia: 8.1 ms por imagen

Williams y Sauber con muchos falsos positivos, por lo que:

- Probamos con nuevos hiperparámetros y aumentamos las epocas.


In [5]:
model.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.20s/it]


                   all         20         29      0.654      0.794      0.815      0.661
           Kick Sauber          4          6      0.794        0.5       0.71      0.508
          Racing Bulls          1          1       0.47          1      0.995      0.796
                Alpine          4          5      0.797      0.787      0.811      0.664
               Ferarri          3          4      0.626       0.75      0.788      0.705
                  Haas          1          1      0.476          1      0.995      0.895
               Mclaren          5          6       0.81        0.5      0.775      0.641
              Mercedes          3          3          1      0.813      0.995      0.735
              Williams          2          3       0.26          1       0.45      0.341
Speed: 2.7ms preprocess, 36.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_optimized32


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002102922D420>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

Comparación y Conclusión
Mejora Global:
La nueva configuración muestra una mejora en el mAP50 (0.818 vs 0.772) y en el mAP50-95 (0.552 vs 0.449), lo que indica que el modelo está detectando con mayor precisión y que sus cajas se ajustan mejor a los objetos reales.

Precisión:
La precisión global ha subido de 0.733 a 0.765, lo que es un buen indicativo de que las predicciones correctas han aumentado.

Recall:
El recall se mantiene prácticamente igual (0.615 vs 0.61), lo que sugiere que el número de falsos negativos no varió significativamente.
Sin embargo, la mejora en mAP50-95 implica que las detecciones precisas han mejorado, aunque quizá se deba trabajar estrategias adicionales para las clases problemáticas (como Kick Sauber, Alpine y Williams).

Hiperparámetros:
El uso de una tasa de aprendizaje variable (con warmup y decaimiento), junto con ajustes más moderados en la intensidad de las transformaciones (reduciendo grados de rotación, mixup y copy_paste) parecen haber contribuido a esta mejora global.

Consideraciones:
Las clases con peor desempeño (por ejemplo, Williams) siguen siendo un reto. Podría ser interesante explorar aumentos específicos o técnicas de fine-tuning para esas clases en particular.

#### Proóximos pasos.

Aumentar a 50 epochs.
Hace fine tuning de las clases problematicas.


# Modelo v3: reentrenar para 250 imágenes

Resultados comparados con el anterior modelo de 200 imágenes:

# Repetir fine tuning pero para v3

Resultados Globales
mAP50:

Anterior: 0.84
Nuevo: 0.871
→ Mejora global.
mAP50-95:

Anterior: 0.728
Nuevo: 0.752
→ También mejora ligeramente.
Por Clase
Kick Sauber:

Anterior: mAP50 = 0.714, mAP50-95 = 0.575
Nuevo: mAP50 = 0.849, mAP50-95 = 0.585
→ Se observa una mejora significativa en mAP50, lo cual es muy positivo.
Racing Bulls:

Ambos modelos mantienen mAP50 muy altos (0.995), con un leve aumento en mAP50-95 (de 0.796 a 0.895).
Alpine:

Anterior: mAP50 = 0.928, mAP50-95 = 0.876
Nuevo: mAP50 = 0.753, mAP50-95 = 0.71
→ Se degrada considerablemente el rendimiento, lo que sugiere que los nuevos parámetros (por ejemplo, una mayor rotación o mixup) podrían estar afectando negativamente a esta clase.
Ferarri:

Una ligera reducción en mAP50 (de 0.845 a 0.808), pero mAP50-95 se mantiene prácticamente igual.
Haas:

Permanece estable (mAP50 ≈ 0.995 y mAP50-95 ≈ 0.895).
Mclaren:

Mejora de mAP50 de 0.805 a 0.903 y un leve incremento en mAP50-95 (de 0.738 a 0.767).
Mercedes:

Se mantiene mAP50 en 0.913, con un aumento en mAP50-95 de 0.726 a 0.793.
Williams:

Mejora de mAP50 de 0.522 a 0.753 y mAP50-95 de 0.473 a 0.627
→ Es un resultado positivo, especialmente si en el YAML se ha bajado el peso de esta clase.
Conclusiones y Recomendaciones
Globalmente, el nuevo fine-tuned mejora las métricas generales (mAP50 y mAP50-95), lo que indica que el modelo, en promedio, está detectando y ajustando mejor las cajas.
Mejoras destacadas en clases problemáticas como Kick Sauber, Mclaren, Mercedes y Williams sugieren que aumentar el dataset a 250 imágenes y ajustar los parámetros (incluyendo la modificación de pesos en el YAML para Williams) ha beneficiado a estas clases.
La principal preocupación es Alpine, cuya precisión (mAP50) se redujo notablemente. Esto podría deberse a:
Los nuevos parámetros de augmentación (por ejemplo, grados de rotación de 15° y mixup de 0.3) podrían estar generando transformaciones que afectan negativamente las características de Alpine.
Es posible que la variabilidad en Alpine no se esté reflejando adecuadamente en el conjunto de datos.
Recomendación:

Es viable continuar con el fine tuning usando 250 imágenes por clase y ajustar el peso de Williams a 1.2 en el YAML, ya que esto ha mostrado mejoras para esa clase.
Sin embargo, sería conveniente revisar o ajustar las transformaciones específicas para Alpine (por ejemplo, disminuir la rotación o el mixup) para intentar recuperar parte del rendimiento perdido en esa clase.
En resumen, el nuevo fine tuning mejora el desempeño global y de varias clases problemáticas, aunque se requiere un ajuste focalizado para Alpine


In [ ]:
# model.save("../f1-strategy/weights/f1_detection_best.pt")
model.save("../f1-strategy/weights/f1_detection_best_2.pt")

In [ ]:
# FIRST DEVELOPED MODEL
# model_fine_tuned.train(
#     data="../f1-strategy/f1-dataset/data.yaml",
#     epochs=NUM_EPOCHS,
#     batch=BATCH_SIZE,
#     optimizer="AdamW",
#     lr0=LEARNING_RATE / 2,  # Aprendizaje más lento para no sobreajustar
#     momentum=0.9,
#     weight_decay=0.0001,
#     label_smoothing=0.1,
#     degrees=10.0,
#     mixup=0.2,
#     patience=5,
#     pretrained=True,  # Importante para mantener el conocimiento previo
#     project='../f1-strategy/yolo-files/runs',
#     name='train_finetune'
# )

In [ ]:
# SECOND DEVELOPED MODEL
# model_fine_tuned.train(
#     data="../f1-strategy/f1-dataset/data.yaml",
#     epochs=NUM_EPOCHS,
#     batch=BATCH_SIZE,
#     optimizer="AdamW",
#     lr0=LEARNING_RATE / 2,  # Mantener un aprendizaje más lento para no sobreajustar
#     momentum=0.9,
#     weight_decay=0.0001,  # Mantener el ajuste original para evitar sobreajuste
#     label_smoothing=0.1,
#     degrees=10.0,
#     mixup=0.2,
#     patience=10,  # Paciencia ajustada a 10
#     pretrained=True,  # Mantener el conocimiento previo
#     project='../f1-strategy/yolo-files/runs',
#     name='train_finetune',
# )

In [ ]:
model_fine_tuned = YOLO("../f1-strategy/weights/f1_detection_best.pt")

model_fine_tuned.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=NUM_EPOCHS,
    batch=BATCH_SIZE,
    optimizer="AdamW",
    lr0=LEARNING_RATE / 3,  # Reducción más agresiva del learning rate
    momentum=0.9,
    weight_decay=0.001,  # Aumentar regularización
    label_smoothing=0.2,  # Mayor suavizado para evitar overconfidence
    degrees=15.0,  # Más rotación para ayudar con ángulos lejanos
    mixup=0.3,  # Mayor mezcla para generalización
    patience=15,  # Más paciencia para convergencia
    pretrained=True,
    project='../f1-strategy/yolo-files/runs',
    name='train_finetune',
    flipud=0.1,  # Añadir flip vertical
    shear=2.0  # Distorsión adicional
)

New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=../f1-strategy/weights/f1_detection_best.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=50, time=None, patience=15, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../f1-strategy/yolo-files/runs, name=train_finetune, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plot

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_finetune\labels.jpg... 
optimizer: AdamW(lr=0.00016666666666666666, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.001), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_finetune
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         29      0.677       0.65      0.782      0.692
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 15, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



30 epochs completed in 0.551 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_finetune\weights\last.pt, 52.0MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_finetune\weights\best.pt, 52.0MB

Validating ..\f1-strategy\yolo-files\runs\train_finetune\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.60it/s]


                   all         20         29      0.699      0.792      0.871      0.752
           Kick Sauber          4          6          1      0.461      0.849      0.585
          Racing Bulls          1          1      0.522          1      0.995      0.895
                Alpine          4          5      0.639      0.716      0.753       0.71
               Ferarri          3          4      0.615       0.75      0.808      0.742
                  Haas          1          1      0.403          1      0.995      0.895
               Mclaren          5          6      0.826      0.796      0.903      0.767
              Mercedes          3          3          1      0.614      0.913      0.793
              Williams          2          3      0.591          1      0.753      0.627
Speed: 0.6ms preprocess, 7.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022CABFEF6D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

#### Best 2 para fine tuned 2:

Comparación con modelo base original:
Modelo Base Original:

mAP50: 0.762
mAP50-95: 0.633

Modelo Base Optimizado:

mAP50: 0.815 (↑ 0.053)
mAP50-95: 0.661 (↑ 0.028)

Análisis por clase:

Kick Sauber:

Original: P=0.784, R=0.5, mAP50=0.691
Optimizado: P=0.794, R=0.5, mAP50=0.71
Mejora: Ligero aumento en precision y mAP, mismo recall

Racing Bulls:

Original: P=0.353, R=1.0, mAP50=0.995
Optimizado: P=0.47, R=1.0, mAP50=0.995
Mejora: Incremento significativo en precision (+0.117)

Alpine:

Original: P=0.532, R=0.6, mAP50=0.669
Optimizado: P=0.797, R=0.787, mAP50=0.811
Mejora: Aumento importante en precision (+0.265) y recall (+0.187)

Ferrari:

Original: P=0.689, R=0.75, mAP50=0.812
Optimizado: P=0.626, R=0.75, mAP50=0.788
Cambio: Ligera reducción en precision, mismo recall

Haas:

Original: P=0.588, R=1.0, mAP50=0.995
Optimizado: P=0.476, R=1.0, mAP50=0.995
Cambio: Reducción en precision, mismo recall

McLaren:

Original: P=1.0, R=0.581, mAP50=0.887
Optimizado: P=0.81, R=0.5, mAP50=0.775
Cambio: Reducción en todos los aspectos, especialmente recall

Mercedes:

Original: P=0.841, R=0.333, mAP50=0.72
Optimizado: P=1.0, R=0.813, mAP50=0.995
Mejora: Gran aumento en recall (+0.48) y mAP, precision perfecta

Williams:

Original: P=0.316, R=1.0, mAP50=0.332
Optimizado: P=0.26, R=1.0, mAP50=0.45
Cambio: Ligera reducción en precision, pero aumento en mAP


In [ ]:
# Cargar el mejor modelo de la etapa anterior
model_fine_tuned_2 = YOLO("../f1-strategy/weights/f1_detection_best_2.pt")


# Fine-tuning con enfoque en mejorar el recall y reducir overfitting
model_fine_tuned_2.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=FINE_TUNED_EPOCHS,
    batch=FINE_TUNED_BATCH_SIZE,
    optimizer="AdamW",      # Mantener optimizador adaptativo
    lr0=LEARNING_RATE/8,    # Learning rate aún más bajo para fine-tuning refinado
    lrf=0.0005,             # Factor LR final más bajo para mejor convergencia
    cos_lr=True,            # Mantener cosine annealing
    momentum=0.937,         # Valor óptimo para AdamW
    weight_decay=0.004,     # Ajustar regularización

    # Balanceo de clases
    label_smoothing=0.1,    # Reducir smoothing para enfocarse más en clases específicas

    # Énfasis en las pérdidas
    box=4.0,                # Ajustar peso para pérdida de bounding box
    cls=3.5,                # Mayor énfasis en clasificación para clases difíciles

    # Augmentaciones ajustadas según resultados
    mosaic=0.9,             # Mayor mosaico para mejorar detección
    mixup=0.3,              # Mantener mixup para generalizar mejor
    copy_paste=0.25,        # Aumentar para más variedad
    degrees=15.0,           # Reducir un poco la rotación
    translate=0.2,          # Mantener traslación
    scale=0.35,             # Mayor variación de escala
    shear=2.5,              # Ajustar distorsión
    perspective=0.001,      # Mantener perspectiva
    fliplr=0.5,             # Mantener flip horizontal
    flipud=0.05,            # Mantener flip vertical ocasional

    # Cambios en color
    hsv_h=0.02,             # Mantener cambios en tono
    hsv_s=0.25,             # Ajustar saturación
    hsv_v=0.15,             # Reducir variación de brillo

    # Parámetros avanzados
    iou=0.65,               # Ajustar umbral IOU

    # Configuración general
    patience=25,            # Mayor paciencia para convergencia
    verbose=True,
    device=DEVICE,
    pretrained=True,
    project='../f1-strategy/yolo-files/runs',
    name='train_finetune_adjusted',

    # Guardar puntos de control
    save_period=2,          # Guardar modelos cada 2 épocas
)

New https://pypi.org/project/ultralytics/8.3.79 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
engine\trainer: task=detect, mode=train, model=../f1-strategy/weights/f1_detection_best_2.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=30, time=None, patience=25, batch=6, imgsz=640, save=True, save_period=2, cache=False, device=cuda, workers=8, project=../f1-strategy/yolo-files/runs, name=train_finetune_adjusted, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.65, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_finetune_adjusted\labels.jpg... 
optimizer: AdamW(lr=6.25e-05, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.004125), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_finetune_adjusted
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         29      0.804      0.841      0.922      0.705



30 epochs completed in 0.593 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_finetune_adjusted\weights\last.pt, 52.0MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_finetune_adjusted\weights\best.pt, 52.0MB

Validating ..\f1-strategy\yolo-files\runs\train_finetune_adjusted\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.80it/s]


                   all         20         29      0.807      0.837      0.921      0.709
           Kick Sauber          4          6          1      0.475       0.73      0.508
          Racing Bulls          1          1      0.529          1      0.995      0.697
                Alpine          4          5          1       0.62       0.92      0.758
               Ferarri          3          4      0.772       0.75      0.912      0.727
                  Haas          1          1      0.598          1      0.995      0.895
               Mclaren          5          6          1      0.965      0.995      0.775
              Mercedes          3          3          1      0.882      0.995      0.688
              Williams          2          3      0.558          1       0.83      0.625
Speed: 0.5ms preprocess, 7.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune_adjusted


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002102BAB8370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [ ]:
# Evaluar después del fine-tuning
model_fine_tuned.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


                   all         20         29        0.7      0.792      0.871      0.752
           Kick Sauber          4          6          1      0.461      0.849      0.585
          Racing Bulls          1          1      0.521          1      0.995      0.895
                Alpine          4          5      0.638      0.715      0.753       0.71
               Ferarri          3          4      0.615       0.75      0.808      0.742
                  Haas          1          1      0.403          1      0.995      0.895
               Mclaren          5          6      0.826      0.797      0.903      0.767
              Mercedes          3          3          1      0.614      0.913      0.793
              Williams          2          3      0.594          1      0.753      0.627
Speed: 2.2ms preprocess, 33.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022CA7200BB0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [8]:
# Después del entrenamiento
model_fine_tuned_2.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


                   all         20         29      0.807      0.837      0.921      0.708
           Kick Sauber          4          6          1      0.475       0.73      0.508
          Racing Bulls          1          1      0.528          1      0.995      0.697
                Alpine          4          5          1      0.626       0.92      0.747
               Ferarri          3          4      0.772       0.75      0.912      0.727
                  Haas          1          1      0.599          1      0.995      0.895
               Mclaren          5          6          1      0.964      0.995      0.778
              Mercedes          3          3          1      0.879      0.995      0.688
              Williams          2          3       0.56          1       0.83      0.625
Speed: 3.8ms preprocess, 29.5ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune_adjusted2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002103D657910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [ ]:
# Guardar el modelo fine-tuned con un nuevo nombre
# model_fine_tuned.save("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")
model_fine_tuned_2.save(
    "../f1-strategy/weights/f1_detection_fine_tuned_best_2.pt")

print("Modelo fine-tuned guardado correctamente en: ../f1-strategy/weights/f1_detection_fine_tuned_best_2.pt")

Modelo fine-tuned guardado correctamente en: ../f1-strategy/weights/f1_detection_fine_tuned_best_2.pt


# Resultados con 200 imágenes

### Conclusiones del modelo fine tuned: Mejor usar el best o el best_fine_tuned?

Resultados del Modelo Anterior:
mAP50: 0.772
mAP50-95: 0.449
Precisión (P): 0.733
Recall (R): 0.615
Clases Problemáticas:

Kick Sauber: mAP50 ~0.619
Alpine: mAP50 ~0.556
Williams: mAP50 ~0.487
Resultados del Nuevo Modelo Fine-Tuned:
En Train:
mAP50: 0.840
mAP50-95: 0.728
Precisión: 0.643
Recall: 0.761
Clases Problemáticas:

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.522
En Validación:
mAP50: 0.839
mAP50-95: 0.729
Precisión: 0.643
Recall: 0.762
Clases Problemáticas:

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.519
Observaciones:
El mAP50 ha mejorado notablemente, subiendo de 0.772 a 0.840 en entrenamiento y 0.839 en validación.
El mAP50-95 también sube de 0.449 a 0.728 en entrenamiento y 0.729 en validación, lo cual indica un aumento en la precisión en diferentes umbrales de IoU.
La precisión ha subido ligeramente de 0.733 a 0.643 en el entrenamiento, pero con un recall mucho más alto, lo que sugiere que el modelo fine-tuned ha mejorado la capacidad de identificar los objetos relevantes, aunque el número de falsos positivos puede haber aumentado.
Las clases problemáticas como Kick Sauber, Alpine, y Williams han mostrado mejoras en mAP50, con un aumento significativo en Alpine (0.928 en validación, frente a 0.556 anteriormente).
Conclusión:
El modelo fine-tuned muestra una mejora general significativa, especialmente en mAP50 y en la capacidad para manejar clases problemáticas, aunque el modelo podría beneficiarse de un ajuste adicional en la precisión, especialmente para clases como Williams. Reducir las epochs de 40 a menos podría funcionar si se optimizan más hiperparámetros, pero parece que las 21 epochs han sido suficientes.

### Siguiente modelo: peso de 2 a williams en el yaml.

Resultados del Nuevo Modelo (Con Peso para Williams):
mAP50: 0.840 en entrenamiento, 0.839 en validación.
mAP50-95: 0.728 en entrenamiento, 0.729 en validación.
Precisión: 0.643 en entrenamiento, 0.643 en validación.
Recall: 0.761 en entrenamiento, 0.762 en validación.
Resultados Anteriores (Sin Ajuste):
mAP50: 0.772 en entrenamiento, 0.772 en validación.
mAP50-95: 0.449 en entrenamiento, 0.449 en validación.
Precisión: 0.733 en entrenamiento, 0.733 en validación.
Recall: 0.615 en entrenamiento, 0.615 en validación.
Clases Problemáticas:
Antes (sin ajuste):

Kick Sauber: mAP50 ~0.619
Alpine: mAP50 ~0.556
Williams: mAP50 ~0.487
Ahora (con ajuste):

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.522
Observaciones:
mAP50 ha mejorado significativamente: de 0.772 a 0.840 en entrenamiento y de 0.772 a 0.839 en validación, lo que refleja un aumento en la capacidad de detectar objetos de manera precisa.
mAP50-95 también ha mejorado considerablemente, subiendo de 0.449 a 0.728 en entrenamiento y 0.449 a 0.729 en validación.
Precisión y Recall en general han subido, pero la precisión ha tenido un ligero descenso (0.733 a 0.643), lo cual podría indicar un mayor número de falsos positivos, aunque el recall ha aumentado, lo que indica una mejora en la identificación de objetos relevantes.
Williams sigue siendo un reto con un mAP50 de 0.522 en entrenamiento y 0.519 en validación, lo que es una ligera mejora respecto al valor anterior de 0.487, pero sigue estando por debajo de otros equipos.


In [ ]:
# Cargar el modelo entrenado para hacer predicciones
# model_fine_tuned_test = YOLO("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")
# model_fine_tuned_test.to(DEVICE)
model_fine_tuned_2_test = YOLO(
    "../f1-strategy/weights/f1_detection_fine_tuned_best_2.pt")

model_fine_tuned_2_test.to(DEVICE)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0-1): 2 x Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=Tr

In [ ]:
# Evaluar en el conjunto de prueba
test_results = model_fine_tuned_2_test.predict(
    source="../f1-strategy/f1-dataset/test/images", save=True)


image 1/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1-2023-Brazilian-GP-FP1-21_jpg.rf.e0a6d6d14a2b145d1285111d65dcf969.jpg: 640x640 1 Ferarri, 119.3ms
image 2/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1019924227-LAT-20231105-GP2320_143500SMG_2193-747_jpg.rf.cd24225e35180d6a3ede83dfb291abea.jpg: 640x640 3 Williamss, 13.4ms
image 3/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\13-sunday-mexicocity-2023_jpg.rf.9fb9bf359354501321e3dbd03e75d0d6.jpg: 640x640 2 Racing Bullss, 1 Aston Martin, 11.8ms
image 4/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\Ferraritop_jpg.rf.2986765a7cdb43c522f47262e9e4b1ab.jpg: 640x640 1 Ferarri, 13.5ms
image 5/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segund

In [ ]:
# Ver el contenido de test_results
print(test_results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Kick Sauber', 1: 'Racing Bulls', 2: 'Alpine', 3: 'Aston Martin', 4: 'Ferarri', 5: 'Haas', 6: 'Mclaren', 7: 'Mercedes', 8: 'Red Bull', 9: 'Williams'}
obb: None
orig_img: array([[[ 32,  39,  12],
        [ 35,  42,  15],
        [ 47,  52,  25],
        ...,
        [233, 220, 222],
        [233, 220, 222],
        [233, 220, 222]],

       [[ 30,  37,  10],
        [ 32,  39,  12],
        [ 42,  47,  20],
        ...,
        [234, 221, 223],
        [233, 220, 222],
        [233, 220, 222]],

       [[ 26,  33,   6],
        [ 28,  35,   8],
        [ 36,  40,  15],
        ...,
        [234, 221, 223],
        [234, 221, 223],
        [233, 220, 222]],

       ...,

       [[ 29,  88,  67],
        [ 27,  89,  65],
        [ 27,  89,  67],
        ...,
        [136, 145, 154],
        [135, 144, 153],
        [134, 143, 152]],

       [[ 

In [ ]:
from ultralytics import YOLO
import torch
import random
import numpy as np

# Definir variables globales
FINE_TUNED_EPOCHS = 40      # Más épocas para un entrenamiento más largo
FINE_TUNED_BATCH_SIZE = 6   # Mantenemos el mismo batch size

# Definir semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# 1. Cargar el modelo base (el original, no el último entrenado)
model_pruned = YOLO('../f1-strategy/weights/f1_detection_fine_tuned_best.pt')

# 2. Configurar el fine-tuning enfocado en clasificación
results = model_pruned.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=FINE_TUNED_EPOCHS,
    imgsz=736,               # Ligeramente más grande para mejor detalle
    batch=FINE_TUNED_BATCH_SIZE,
    patience=12,             # Mayor paciencia para early stopping
    cos_lr=True,
    lr0=0.0001,              # Learning rate inicial MENOR
    lrf=0.000005,            # Learning rate final MENOR
    optimizer='AdamW',
    weight_decay=0.001,      # MAYOR regularización
    warmup_epochs=5,         # Más epochs de warmup

    # Parámetros optimizados para clasificación a distancia
    scale=0.7,               # MAYOR variación de escalado
    degrees=3.0,             # Menos rotación
    shear=1.5,               # Menos deformación
    perspective=0.0003,
    flipud=0.01,
    fliplr=0.4,              # Más flip horizontal
    mosaic=0.9,              # MAYOR mosaico
    mixup=0.15,              # MAYOR mixup
    copy_paste=0.2,          # MAYOR copy-paste

    # Ajustes para diferentes condiciones de luz
    hsv_h=0.02,
    hsv_s=0.8,               # MAYOR variación de saturación
    hsv_v=0.6,               # MAYOR variación de brillo

    # AJUSTE CRUCIAL: Mayor peso a la pérdida de clasificación
    box=6.0,
    cls=5.0,                 # MAYOR énfasis en clasificación
    dfl=1.5,

    # Otros parámetros
    overlap_mask=True,
    mask_ratio=4,
    iou=0.5,                 # IoU threshold REDUCIDO (más tolerante)
    single_cls=False,
    nbs=64,
    close_mosaic=15,
    rect=False,
    cache=False,

    val=True,
    save=True,
    save_period=5,
    plots=True,

    amp=True,

    project='../f1-strategy/yolo-files/runs',
    name='train_cls_focused',
    exist_ok=True
)

New https://pypi.org/project/ultralytics/8.3.79 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=../f1-strategy/weights/f1_detection_fine_tuned_best.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=40, time=None, patience=12, batch=6, imgsz=736, save=True, save_period=5, cache=False, device=None, workers=8, project=../f1-strategy/yolo-files/runs, name=train_cls_focused, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=15, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fa

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_cls_focused\labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00103125), 83 bias(decay=0.0)
Image sizes 736 train, 736 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_cls_focused
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      31/40         4G     0.4158      2.665       1.09          4        736: 100%|██████████| 417/417 [01:25<00:00,  4.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.96it/s]

                   all         20         29       0.64      0.718      0.824      0.696
EarlyStopping: Training stopped early as no improvement observed in last 12 epochs. Best results observed at epoch 19, best model saved as best.pt.
To update EarlyStopping(patience=12) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



31 epochs completed in 0.770 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_cls_focused\weights\last.pt, 52.1MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_cls_focused\weights\best.pt, 52.1MB

Validating ..\f1-strategy\yolo-files\runs\train_cls_focused\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.66it/s]


                   all         20         29      0.742      0.764      0.892       0.75
           Kick Sauber          4          6          1      0.783      0.909      0.649
          Racing Bulls          1          1       0.34          1      0.995      0.796
                Alpine          4          5      0.712        0.8      0.938      0.849
               Ferarri          3          4       0.76       0.75      0.888       0.78
                  Haas          1          1      0.655          1      0.995      0.895
               Mclaren          5          6          1      0.578      0.942      0.788
              Mercedes          3          3          1      0.591      0.863      0.691
              Williams          2          3      0.472       0.61       0.61      0.554
Speed: 1.1ms preprocess, 9.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_cls_focused


In [ ]:
# Evaluar el modelo resultante
print("\nEvaluando modelo con enfoque en clasificación...")
metrics = model_pruned.val()

print(f"\nEntrenamiento completado. Resultados:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")


Evaluando modelo con enfoque en clasificación...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


                   all         20         29      0.742      0.764      0.892       0.75
           Kick Sauber          4          6          1      0.784      0.909      0.649
          Racing Bulls          1          1      0.339          1      0.995      0.796
                Alpine          4          5      0.709        0.8      0.938      0.849
               Ferarri          3          4       0.76       0.75      0.888       0.78
                  Haas          1          1      0.655          1      0.995      0.895
               Mclaren          5          6          1      0.578      0.942      0.788
              Mercedes          3          3          1      0.591      0.863      0.691
              Williams          2          3      0.472      0.612       0.61      0.554
Speed: 3.1ms preprocess, 32.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_cls_focused

Entrenamiento completado. Resultados:
mAP50: 0.89

In [22]:
model_pruned.save("../f1-strategy/weights/model_pruned_best.pt")

In [24]:
model_pruned_test = YOLO("../f1-strategy/weights/model_pruned_best.pt")

model_pruned_test.to(DEVICE)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0-1): 2 x Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=Tr

In [ ]:
# Evaluar en el conjunto de prueba
test_results_pruned = model_pruned_test.predict(
    source="../f1-strategy/f1-dataset/test/images", save=True)

print(test_results_pruned)


image 1/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1-2023-Brazilian-GP-FP1-21_jpg.rf.e0a6d6d14a2b145d1285111d65dcf969.jpg: 736x736 1 Ferarri, 105.0ms
image 2/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1019924227-LAT-20231105-GP2320_143500SMG_2193-747_jpg.rf.cd24225e35180d6a3ede83dfb291abea.jpg: 736x736 1 Kick Sauber, 1 Red Bull, 2 Williamss, 18.9ms
image 3/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\13-sunday-mexicocity-2023_jpg.rf.9fb9bf359354501321e3dbd03e75d0d6.jpg: 736x736 1 Racing Bulls, 1 Aston Martin, 18.1ms
image 4/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\Ferraritop_jpg.rf.2986765a7cdb43c522f47262e9e4b1ab.jpg: 736x736 1 Ferarri, 18.7ms
image 5/11 c:\Users\victo\Desktop\

In [4]:
from ultralytics import YOLO
import torch
import random
import numpy as np
import os
import yaml
import shutil
from pathlib import Path

# Definir semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# Variables de configuración
FINE_TUNED_EPOCHS = 50      # Más épocas todavía
FINE_TUNED_BATCH_SIZE = 4   # Batch size reducido para mayor precisión
PATIENCE = 15               # Mayor paciencia para early stopping

# Directorios principales
BASE_DIR = '../f1-strategy'
DATASET_DIR = os.path.join(BASE_DIR, 'f1-dataset')
DATA_YAML = os.path.join(DATASET_DIR, 'data.yaml')

# Paso 1: Modificar el archivo data.yaml para agregar ponderación de clases
# Esto es crucial para resolver el problema de Williams
def update_data_yaml_with_class_weights():
    # Leer el archivo data.yaml existente
    with open(DATA_YAML, 'r') as f:
        data_config = yaml.safe_load(f)

    # Hacer una copia de seguridad del original
    backup_file = DATA_YAML + '.backup'
    if not os.path.exists(backup_file):
        shutil.copy(DATA_YAML, backup_file)
        print(f"Backup creado en {backup_file}")

    # Obtener nombres de clases
    class_names = data_config.get('names', [])

    # Crear pesos personalizados por clase (mayor peso a Williams y clases problemáticas)
    class_weights = {}
    for i, name in enumerate(class_names):
        if name.lower() == 'williams':
            class_weights[i] = 2.0  # Mayor peso a Williams
        elif name.lower() in ['alpine', 'red bull']:
            class_weights[i] = 1.5  # Mayor peso a clases confundidas
        else:
            class_weights[i] = 1.0  # Peso normal para el resto

    # Agregar pesos a la configuración
    data_config['class_weights'] = class_weights

    # Guardar el archivo actualizado
    with open(DATA_YAML, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)

    print(f"Archivo {DATA_YAML} actualizado con pesos de clase:")
    for name, weight in zip(class_names, [class_weights.get(i, 1.0) for i in range(len(class_names))]):
        print(f"  - {name}: {weight}")

    return data_config

# Actualizar data.yaml con pesos de clase
data_config = update_data_yaml_with_class_weights()

# Cargar el modelo base
model_pruned_2 = YOLO('../f1-strategy/weights/yolov8m.pt')  # USAR MODELO BASE

print("\n=== CONFIGURACIÓN DEL ENTRENAMIENTO ===")
print(f"Épocas: {FINE_TUNED_EPOCHS}")
print(f"Batch size: {FINE_TUNED_BATCH_SIZE}")
print(f"Paciencia: {PATIENCE}")
print(f"Peso de clasificación: 7.0 (muy alto)")
print(f"Williams ponderado 2.0 veces más")
print(f"Learning rate: 0.00008 (muy bajo para aprendizaje preciso)")
print("Augmentación extrema para objetos a distancia")
print("===================================\n")

# Pregunta de confirmación
input("Presiona Enter para comenzar el entrenamiento, o Ctrl+C para cancelar...")

# Configurar el entrenamiento desde cero con hiperparámetros personalizados directamente
results = model_pruned_2.train(
    data=DATA_YAML,
    epochs=FINE_TUNED_EPOCHS,
    imgsz=800,                # Tamaño de imagen MUCHO mayor para detalles
    batch=FINE_TUNED_BATCH_SIZE,
    patience=PATIENCE,        # Mayor paciencia
    
    # Hiperparámetros de pérdida directamente como argumentos
    box=6.0,             # Peso de pérdida de bounding box
    cls=7.0,             # Peso de pérdida de clasificación (MUY ALTO)
    dfl=1.5,             # Peso de pérdida DFL
    
    # Argumentos para aumentación extrema para objetos a distancia
    scale=0.8,           # Variación de escala extrema
    flipud=0.05,         # Más flip vertical
    fliplr=0.5,          # Más flip horizontal
    mosaic=0.95,         # Mosaico casi siempre
    mixup=0.25,          # Mixup frecuente
    copy_paste=0.3,      # Copy-paste frecuente
    degrees=2.5,         # Rotación moderada
    translate=0.2,       # Traslación significativa
    shear=1.0,           # Menos deformación para preservar características
    perspective=0.0001,  # Efecto de perspectiva mínimo
    hsv_h=0.015,         # Variación de tono
    hsv_s=0.9,           # Variación de saturación extrema
    hsv_v=0.8,           # Variación de brillo extrema
    
    # Optimizador
    optimizer='AdamW',   # Optimizador
    lr0=0.00008,         # Learning rate inicial (MUY BAJO)
    lrf=0.000001,        # Learning rate final (MUY BAJO)
    momentum=0.937,      # Momentum
    weight_decay=0.002,  # Mayor regularización
    warmup_epochs=8.0,   # Más épocas de warmup
    warmup_momentum=0.8, # Momentum de warmup
    warmup_bias_lr=0.1,  # Learning rate de bias en warmup
    
    # Parámetros de regularización adicionales
    label_smoothing=0.1, # Suavizado de etiquetas para mejor generalización

    # Otros parámetros de entrenamiento
    cache=False,
    device='0',           # Usar primera GPU
    workers=8,            # Máximo de workers
    
    # Validación y guardado
    val=True,
    save=True,
    save_period=5,
    plots=True,
    
    # Mixed precision para rendimiento
    amp=True,
    
    # Configuración del proyecto
    project='../f1-strategy/yolo-files/runs',
    name='train_extreme_cls',
    exist_ok=True
)



Archivo ../f1-strategy\f1-dataset\data.yaml actualizado con pesos de clase:
  - Kick Sauber: 1.0
  - Racing Bulls: 1.0
  - Alpine: 1.5
  - Aston Martin: 1.0
  - Ferarri: 1.0
  - Haas: 1.0
  - Mclaren: 1.0
  - Mercedes: 1.0
  - Red Bull: 1.5
  - Williams: 2.0

=== CONFIGURACIÓN DEL ENTRENAMIENTO ===
Épocas: 50
Batch size: 4
Paciencia: 15
Peso de clasificación: 7.0 (muy alto)
Williams ponderado 2.0 veces más
Learning rate: 0.00008 (muy bajo para aprendizaje preciso)
Augmentación extrema para objetos a distancia

New https://pypi.org/project/ultralytics/8.3.80 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=../f1-strategy/weights/yolov8m.pt, data=../f1-strategy\f1-dataset\data.yaml, epochs=50, time=None, patience=15, batch=4, imgsz=800, save=True, save_peri

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_extreme_cls\labels.jpg... 
optimizer: AdamW(lr=8e-05, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.002), 83 bias(decay=0.0)
Image sizes 800 train, 800 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_extreme_cls
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      32/50      3.32G     0.6706      7.776      1.272         10        800: 100%|██████████| 625/625 [01:44<00:00,  6.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  9.06it/s]

                   all         20         29      0.788      0.848       0.96      0.795
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 17, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



32 epochs completed in 0.942 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_extreme_cls\weights\last.pt, 52.1MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_extreme_cls\weights\best.pt, 52.1MB

Validating ..\f1-strategy\yolo-files\runs\train_extreme_cls\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  8.24it/s]


                   all         20         29       0.69      0.856      0.956      0.801
           Kick Sauber          4          6          1      0.819       0.86       0.62
          Racing Bulls          1          1      0.223          1      0.995      0.796
                Alpine          4          5      0.689        0.8       0.92      0.805
               Ferarri          3          4      0.855          1      0.995       0.91
                  Haas          1          1      0.581          1      0.995      0.895
               Mclaren          5          6          1      0.287      0.972      0.794
              Mercedes          3          3      0.735      0.939      0.913      0.691
              Williams          2          3      0.436          1      0.995      0.895
Speed: 0.7ms preprocess, 11.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_extreme_cls


class loss en este caso está artificialmente subido por el peso de 7, por lo que de momento durante el entrenamiento ya es sustancialmente mejor que los anteriores si hacemos la conversion (8.2/7 aprox 1.2 frente a 2.6 del anterior).

Por tanto, parece que está empezando a funcionar mejor pero habrá que verlo una vez acabe el entrenamiento.

In [5]:
# Evaluar el modelo resultante
print("\nEvaluando modelo con enfoque extremo en clasificación...")
metrics = model_pruned_2.val()

print(f"\nEntrenamiento completado. Resultados:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

# Guardar el modelo final
model_pruned_2.save("../f1-strategy/weights/model_extreme_cls.pt")
print("Modelo guardado en: ../f1-strategy/weights/model_extreme_cls.pt")

print("\n=== RECOMENDACIONES PARA USO DEL MODELO ===")
print("1. Este modelo se entrenó con pesos de clase para Williams")
print("2. Use umbrales de confianza específicos por clase en video:")
print("   - Williams: 0.60")
print("   - Alpine: 0.45")
print("   - Red Bull: 0.40")
print("   - Otros: 0.35")
print("3. En objetos lejanos, ignore detecciones Williams con baja confianza")
print("===================================")


Evaluando modelo con enfoque extremo en clasificación...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.35it/s]


                   all         20         29      0.688      0.856      0.958      0.803
           Kick Sauber          4          6          1      0.819       0.86      0.622
          Racing Bulls          1          1      0.222          1      0.995      0.796
                Alpine          4          5      0.682        0.8      0.938       0.82
               Ferarri          3          4      0.854          1      0.995       0.91
                  Haas          1          1      0.579          1      0.995      0.895
               Mclaren          5          6          1      0.291      0.972      0.794
              Mercedes          3          3      0.735      0.942      0.913      0.691
              Williams          2          3      0.435          1      0.995      0.895
Speed: 1.1ms preprocess, 29.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_extreme_cls

Entrenamiento completado. Resultados:
mAP50: 0.95

In [9]:
model_extreme_test = YOLO("../f1-strategy/weights/model_extreme_cls.pt")

model_extreme_test.to(DEVICE)

# Evaluar en el conjunto de prueba
test_results_pruned = model_extreme_test.predict(
    source="../f1-strategy/f1-dataset/test/images", save=True)

print(test_results_pruned)


image 1/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1-2023-Brazilian-GP-FP1-21_jpg.rf.e0a6d6d14a2b145d1285111d65dcf969.jpg: 800x800 1 Ferarri, 28.4ms
image 2/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1019924227-LAT-20231105-GP2320_143500SMG_2193-747_jpg.rf.cd24225e35180d6a3ede83dfb291abea.jpg: 800x800 4 Williamss, 24.6ms
image 3/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\13-sunday-mexicocity-2023_jpg.rf.9fb9bf359354501321e3dbd03e75d0d6.jpg: 800x800 1 Racing Bulls, 1 Aston Martin, 21.7ms
image 4/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\Ferraritop_jpg.rf.2986765a7cdb43c522f47262e9e4b1ab.jpg: 800x800 1 Ferarri, 20.8ms
image 5/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo 

Análisis de Resultados del Nuevo Modelo
¡Excelente! Los resultados son significativamente mejores que todos los modelos anteriores. Vamos a hacer una comparación detallada:
Métricas Generales
MétricaModelo OriginalSegundo ModeloModelo ClasificaciónNuevo ModelomAP500.8710.9210.8920.958 ✅mAP50-950.7520.7090.7500.803 ✅
El nuevo modelo supera a todos los anteriores en ambas métricas clave.
Específicamente para Williams (el problema principal)
Métrica WilliamsModelo AnteriorNuevo ModeloPrecisión0.4720.435Recall0.6121.000 ✅mAP500.6100.995 ✅mAP50-950.5540.895 ✅

### Modelo fine tuned del extreme

In [1]:
from ultralytics import YOLO
import torch
import random
import numpy as np
import os
import yaml
import shutil

# Definir semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# Variables de configuración - Fine-tuning más corto
FINE_TUNED_EPOCHS = 25      # Algunas épocas más para asegurar la corrección
FINE_TUNED_BATCH_SIZE = 6   # Batch size reducido para mayor precisión  
PATIENCE = 12               # Mayor paciencia para asegurar convergencia

# Directorios principales
BASE_DIR = '../f1-strategy'
DATASET_DIR = os.path.join(BASE_DIR, 'f1-dataset')
DATA_YAML = os.path.join(DATASET_DIR, 'data.yaml')

# Paso 1: Modificar el archivo data.yaml para ajustar ponderación de clases
# ¡AJUSTE DRÁSTICO PARA ALPINE!
def update_data_yaml_with_extreme_weights():
    # Leer el archivo data.yaml existente
    with open(DATA_YAML, 'r') as f:
        data_config = yaml.safe_load(f)

    # Hacer una copia de seguridad del original
    backup_file = DATA_YAML + '.backup2'
    if not os.path.exists(backup_file):
        shutil.copy(DATA_YAML, backup_file)
        print(f"Backup creado en {backup_file}")

    # Obtener nombres de clases
    class_names = data_config.get('names', [])

    # Crear pesos EXTREMADAMENTE desequilibrados por clase para corregir
    class_weights = {}
    for i, name in enumerate(class_names):
        if name.lower() == 'williams':
            class_weights[i] = 1.5  # Mantener peso razonable (era 2.0)
        elif name.lower() == 'alpine':
            class_weights[i] = 0.3  # REDUCCIÓN EXTREMA para Alpine
        elif name.lower() == 'mclaren':
            class_weights[i] = 2.5  # Mayor peso para McLaren
        else:
            class_weights[i] = 1.0  # Peso normal para el resto

    # Agregar pesos a la configuración
    data_config['class_weights'] = class_weights

    # Guardar el archivo actualizado
    with open(DATA_YAML, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)

    print(f"Archivo {DATA_YAML} actualizado con pesos EXTREMOS para clases:")
    for name, weight in zip(class_names, [class_weights.get(i, 1.0) for i in range(len(class_names))]):
        print(f"  - {name}: {weight}")

    return data_config

# Actualizar data.yaml con pesos extremos
data_config = update_data_yaml_with_extreme_weights()

# Cargar el modelo extreme_cls que ya funciona bien
model_fine_extreme = YOLO('../f1-strategy/weights/model_extreme_cls.pt')

print("\n=== CONFIGURACIÓN DEL FINE-TUNING ANTI-ALPINE ===")
print(f"Épocas: {FINE_TUNED_EPOCHS}")
print(f"Batch size: {FINE_TUNED_BATCH_SIZE}")
print(f"Paciencia: {PATIENCE}")
print(f"Peso de Williams: 1.5 (reducido)")
print(f"Peso de Alpine: 0.3 (DRÁSTICAMENTE reducido)")
print(f"Peso de McLaren: 2.5 (aumentado significativamente)")
print("===================================\n")

# Pregunta de confirmación
input("Presiona Enter para comenzar el fine-tuning ANTI-ALPINE, o Ctrl+C para cancelar...")

# Configurar el fine-tuning con enfoque anti-Alpine
results = model_fine_extreme.train(
    data=DATA_YAML,
    epochs=FINE_TUNED_EPOCHS,
    imgsz=800,                # Mantener mismo tamaño
    batch=FINE_TUNED_BATCH_SIZE,
    patience=PATIENCE,
    
    # Hiperparámetros de pérdida 
    box=6.0,                  # Mantener igual
    cls=9.0,                  # AUMENTADO para dar más importancia a clasificación
    dfl=1.5,                  # Mantener igual
    
    # Parámetros de aumentación específicos
    scale=0.8,
    flipud=0.05,
    fliplr=0.5,
    mosaic=0.9,               # Alto para más variedad
    mixup=0.3,                # Aumentado para más robustez
    copy_paste=0.3,
    degrees=2.5,
    translate=0.2,           
    shear=0.9,                
    perspective=0.0002,      # Ligeramente aumentado
    hsv_h=0.02,
    hsv_s=0.9,                # Aumentado para más variabilidad
    hsv_v=0.7,                
    
    # Optimizador con learning rate bajo pero no demasiado
    optimizer='AdamW',
    lr0=0.00007,              # Ligeramente más alto que antes
    lrf=0.000001,
    momentum=0.937,
    weight_decay=0.003,       # Mayor regularización para evitar sobreajuste a Alpine
    warmup_epochs=4.0,        
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    
    # Mayor regularización para combatir sobreajuste
    label_smoothing=0.15,     # Aumentado significativamente

    # Otros parámetros
    cache=False,
    device='0',
    workers=8,
    
    # Validación y guardado
    val=True,
    save=True,
    save_period=2,            # Guardar más frecuentemente
    plots=True,
    
    # Mixed precision
    amp=True,
    
    # Configuración del proyecto
    project='../f1-strategy/yolo-files/runs',
    name='anti_alpine_tuning',
    exist_ok=True
)



Backup creado en ../f1-strategy\f1-dataset\data.yaml.backup2
Archivo ../f1-strategy\f1-dataset\data.yaml actualizado con pesos EXTREMOS para clases:
  - Kick Sauber: 1.0
  - Racing Bulls: 1.0
  - Alpine: 0.3
  - Aston Martin: 1.0
  - Ferarri: 1.0
  - Haas: 1.0
  - Mclaren: 2.5
  - Mercedes: 1.0
  - Red Bull: 1.0
  - Williams: 1.5

=== CONFIGURACIÓN DEL FINE-TUNING ANTI-ALPINE ===
Épocas: 25
Batch size: 6
Paciencia: 12
Peso de Williams: 1.5 (reducido)
Peso de Alpine: 0.3 (DRÁSTICAMENTE reducido)
Peso de McLaren: 2.5 (aumentado significativamente)

New https://pypi.org/project/ultralytics/8.3.80 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=../f1-strategy/weights/model_extreme_cls.pt, data=../f1-strategy\f1-dataset\data.yaml, epochs=25, time=None, patien

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2500/2500 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\anti_alpine_tuning\labels.jpg... 
optimizer: AdamW(lr=7e-05, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00309375), 83 bias(decay=0.0)
Image sizes 800 train, 800 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\anti_alpine_tuning
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      23/25      4.34G     0.5548      6.582      1.286          5        800: 100%|██████████| 417/417 [01:39<00:00,  4.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.64it/s]

                   all         20         29      0.861      0.748      0.936      0.751
EarlyStopping: Training stopped early as no improvement observed in last 12 epochs. Best results observed at epoch 11, best model saved as best.pt.
To update EarlyStopping(patience=12) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



23 epochs completed in 0.651 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\anti_alpine_tuning\weights\last.pt, 52.1MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\anti_alpine_tuning\weights\best.pt, 52.1MB

Validating ..\f1-strategy\yolo-files\runs\anti_alpine_tuning\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.61it/s]


                   all         20         29      0.815      0.819      0.958      0.756
           Kick Sauber          4          6          1      0.778       0.85      0.651
          Racing Bulls          1          1      0.463          1      0.995      0.697
                Alpine          4          5      0.979          1      0.995      0.836
               Ferarri          3          4          1       0.83      0.995      0.909
                  Haas          1          1      0.713          1      0.995      0.796
               Mclaren          5          6          1      0.278      0.927      0.614
              Mercedes          3          3      0.935      0.667      0.913      0.685
              Williams          2          3       0.43          1      0.995      0.863
Speed: 0.3ms preprocess, 12.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\anti_alpine_tuning


In [2]:
# Evaluar el modelo resultante
print("\nEvaluando modelo con fine-tuning ANTI-ALPINE...")
metrics = model_fine_extreme.val()

print(f"\nFine-tuning completado. Resultados:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

# Guardar el modelo final
model_fine_extreme.save("../f1-strategy/weights/model_anti_alpine.pt")
print("Modelo guardado en: ../f1-strategy/weights/model_anti_alpine.pt")

print("\n=== RECOMENDACIONES PARA USO DEL MODELO ===")
print("1. Este modelo ha sido fine-tuned específicamente para reducir sobredetecciones de Alpine")
print("2. Use umbrales de confianza ajustados en video:")
print("   - Williams: 0.50")  # Ligeramente reducido
print("   - Alpine: 0.75")    # MUY ALTO para evitar falsos positivos
print("   - McLaren: 0.30")   # Bajo para mejorar detección
print("   - Otros: 0.40")
print("3. Para Alpine, considerar también filtrado por tamaño/proporción para reducir falsos positivos")
print("===================================")


Evaluando modelo con fine-tuning ANTI-ALPINE...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


                   all         20         29      0.815      0.819      0.958      0.754
           Kick Sauber          4          6          1      0.778       0.85      0.651
          Racing Bulls          1          1      0.463          1      0.995      0.697
                Alpine          4          5       0.98          1      0.995      0.836
               Ferarri          3          4          1      0.831      0.995      0.909
                  Haas          1          1      0.713          1      0.995      0.796
               Mclaren          5          6          1      0.278      0.927      0.595
              Mercedes          3          3      0.937      0.667      0.913      0.685
              Williams          2          3       0.43          1      0.995      0.863
Speed: 2.6ms preprocess, 38.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\anti_alpine_tuning

Fine-tuning completado. Resultados:
mAP50: 0.958

In [5]:
model_anti_alpine = YOLO("../f1-strategy/weights/model_anti_alpine.pt")

model_anti_alpine.to(DEVICE)

# Evaluar en el conjunto de prueba
test_results_anti_alpine = model_anti_alpine.predict(
    source="../f1-strategy/f1-dataset/test/images", save=True)

print(test_results_anti_alpine)


image 1/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1-2023-Brazilian-GP-FP1-21_jpg.rf.e0a6d6d14a2b145d1285111d65dcf969.jpg: 800x800 1 Ferarri, 19.8ms
image 2/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\1019924227-LAT-20231105-GP2320_143500SMG_2193-747_jpg.rf.cd24225e35180d6a3ede83dfb291abea.jpg: 800x800 1 Kick Sauber, 2 Williamss, 19.5ms
image 3/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\13-sunday-mexicocity-2023_jpg.rf.9fb9bf359354501321e3dbd03e75d0d6.jpg: 800x800 1 Racing Bulls, 1 Aston Martin, 20.4ms
image 4/11 c:\Users\victo\Desktop\Documents\Tercer ao\Segundo Cuatrimestre\Finales\scripts\..\f1-strategy\f1-dataset\test\images\Ferraritop_jpg.rf.2986765a7cdb43c522f47262e9e4b1ab.jpg: 800x800 1 Ferarri, 21.6ms
image 5/11 c:\Users\victo\Desktop\Documents\Ter

In [ ]:
from ultralytics import YOLO
import torch
import random
import numpy as np
import os
import yaml
import shutil

# Definir semilla para reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# Variables de configuración - Entrenamiento completo
EPOCHS = 50
BATCH_SIZE = 8
PATIENCE = 20

# Directorios principales
BASE_DIR = '../f1-strategy'
DATASET_DIR = os.path.join(BASE_DIR, 'f1-dataset')
DATA_YAML = os.path.join(DATASET_DIR, 'data.yaml')

# Modificar el archivo data.yaml con ponderaciones perfectamente equilibradas
def update_data_yaml_with_perfect_balance():
    # Leer el archivo data.yaml existente
    with open(DATA_YAML, 'r') as f:
        data_config = yaml.safe_load(f)

    # Hacer una copia de seguridad del original
    backup_file = DATA_YAML + '.perfect_backup'
    if not os.path.exists(backup_file):
        shutil.copy(DATA_YAML, backup_file)
        print(f"Backup creado en {backup_file}")

    # Obtener nombres de clases
    class_names = data_config.get('names', [])

    # Crear pesos PERFECTAMENTE EQUILIBRADOS
    # Damos peso 1.0 a TODAS las clases para eliminar sesgos
    class_weights = {}
    for i, name in enumerate(class_names):
        # Todas las clases con el mismo peso exacto
        class_weights[i] = 1.0

    # Agregar pesos a la configuración
    data_config['class_weights'] = class_weights

    # Guardar el archivo actualizado
    with open(DATA_YAML, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)

    print(f"Archivo {DATA_YAML} actualizado con pesos perfectamente equilibrados:")
    for name, weight in zip(class_names, [class_weights.get(i, 1.0) for i in range(len(class_names))]):
        print(f"  - {name}: {weight}")

    return data_config

# Actualizar data.yaml con pesos perfectamente equilibrados
data_config = update_data_yaml_with_perfect_balance()

# Cargar el perfect_modelo base YOLOv8m para entrenamiento desde cero
perfect_model = YOLO('../f1-strategy/weights/yolov8m.pt')

print("\n=== CONFIGURACIÓN DEL ENTRENAMIENTO CON BALANCE PERFECTO ===")
print(f"Épocas: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Paciencia: {PATIENCE}")
print(f"Todas las clases con peso exacto 1.0")
print("Énfasis en equilibrio absoluto")
print("===================================\n")

# Pregunta de confirmación
input("Presiona Enter para comenzar el entrenamiento, o Ctrl+C para cancelar...")

# Configurar el entrenamiento desde cero con balance perfecto
results = perfect_model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=800,               # Tamaño de imagen grande para detalles
    batch=BATCH_SIZE,
    patience=PATIENCE,
    
    # Hiperparámetros de pérdida equilibrados
    box=7.0,                 # Buen peso para box
    cls=7.0,                 # Igual peso para clasificación
    dfl=1.5,                 # Mantener igual
    
    # Parámetros de aumentación estándar (no extremos)
    scale=0.7,
    flipud=0.05,
    fliplr=0.5,
    mosaic=0.8,             
    mixup=0.2,
    copy_paste=0.2,
    degrees=2.0,
    translate=0.15,
    shear=0.7,
    perspective=0.0001,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.5,
    
    # Optimizador con tasa de aprendizaje baja
    optimizer='AdamW',
    lr0=0.0001,
    lrf=0.00001,
    momentum=0.937,
    weight_decay=0.001,
    warmup_epochs=5.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    
    # Parámetros de regularización  
    label_smoothing=0.05,

    # Otros parámetros
    cache=False,
    device='0',
    workers=8,
    
    # Validación y guardado
    val=True,
    save=True,
    save_period=5,
    plots=True,
    
    # Mixed precision
    amp=True,
    
    # Configuración del proyecto
    project='../f1-strategy/yolo-files/runs',
    name='perfect_balance',
    exist_ok=True
)



In [ ]:
# Evaluar el perfect_modelo resultante
print("\nEvaluando perfect_modelo con balance perfecto...")
metrics = perfect_model.val()

print(f"\nEntrenamiento completado. Resultados:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

# Guardar el perfect_modelo final
perfect_model.save("../f1-strategy/weights/perfect_balance_perfect_model.pt")
print("perfect_modelo guardado en: ../f1-strategy/weights/perfect_balance_perfect_model.pt")

print("\n=== RECOMENDACIONES PARA USO DEL perfect_modelO ===")
print("1. Este perfect_modelo tiene balance perfecto entre todas las clases")
print("2. Use umbrales de confianza IGUALES en video:")
print("   - Todas las clases: 0.45")
print("   - Para objetos lejanos: 0.50")
print("3. Enfocarse en el post-procesamiento temporal más que en umbrales por clase")
print("===================================")

### Detección actualizada

In [ ]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
from collections import Counter

# Cargar el modelo original
model = YOLO('../f1-strategy/weights/model_extreme_cls.pt')

def process_video(video_path, output_path=None, show=True):
    # Abrir video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return

    # Parámetros del video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Configurar salida si se solicita
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    # Variables para seguimiento temporal
    last_detections = {}  # Para el tracking temporal
    track_history = {}    # Historial de posiciones
    id_counter = 0        # Contador para IDs de objetos

    # Historial de clases para cada objeto
    class_history = {}

    # Color por equipo real (no por predicción)
    team_colors = {
        'Ferrari': (0, 0, 255),       # Rojo
        'Mercedes': (192, 192, 192),  # Plateado
        'Red Bull': (0, 0, 139),      # Azul oscuro
        'McLaren': (255, 165, 0),     # Naranja
        'Aston Martin': (0, 128, 0),  # Verde
        'Alpine': (0, 0, 128),        # Azul
        'Williams': (0, 0, 205),      # Azul claro
        'Haas': (255, 255, 255),      # Blanco
        'Kick Sauber': (0, 255, 255), # Cian
        'Racing Bulls': (255, 0, 0),  # Azul
        'background': (128, 128, 128) # Gris
    }

    # UMBRAL MUY ESTRICTO PARA ALPINE
    class_thresholds = {
        'Williams': 0.70,     # Alto para Williams
        'Alpine': 0.98,       # EXTREMADAMENTE alto para Alpine
        'McLaren': 0.30,      # Bajo para McLaren
        'Red Bull': 0.40,
        'Ferrari': 0.40,
        'Mercedes': 0.40,
        'Haas': 0.40,
        'Kick Sauber': 0.40,
        'Racing Bulls': 0.40,
        'Aston Martin': 0.40,
        'background': 0.50
    }

    # Variables para evaluar el tamaño de los objetos
    size_categories = {
        'small': 5000,    # Área en píxeles² (objetos lejanos)
        'medium': 15000,  # Objetos a media distancia
        'large': 30000    # Objetos cercanos
    }

    # Variables para calcular FPS real
    frame_count = 0
    start_time = time.time()
    fps_update_interval = 10  # Actualizar FPS cada 10 frames
    current_fps = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame_count += 1
        original_frame = frame.copy()

        # Calcular FPS real periódicamente
        if frame_count % fps_update_interval == 0:
            end_time = time.time()
            current_fps = fps_update_interval / (end_time - start_time)
            start_time = time.time()

        # DETECCIÓN YOLO
        results = model.predict(
            source=frame,
            conf=0.30,       # Umbral base
            iou=0.45,
            max_det=20,
            classes=None,
            agnostic_nms=False,
            verbose=False,
            stream=True,
            vid_stride=1
        )

        # Objetos detectados en frame actual
        current_detections = {}

        # Procesar resultados
        for r in results:
            boxes = r.boxes

            if len(boxes) > 0:
                # Recorrer todas las detecciones
                for box in boxes:
                    # Extraer información
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    cls_name = model.names[cls_id]
                    
                    # Calcular centro para tracking
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2

                    # Área de la detección
                    area = (x2 - x1) * (y2 - y1)

                    # Ajustar umbral de confianza según el tamaño y la clase
                    threshold = class_thresholds.get(cls_name, 0.40)

                    # Aplicar ajustes basados en tamaño
                    if area < size_categories['small']:
                        if cls_name == 'Alpine':
                            threshold = 0.99  # EXTREMADAMENTE alto para Alpine lejano
                        elif cls_name == 'Williams':
                            threshold = 0.80  # Alto para Williams lejano

                    # Solo procesar si supera el umbral ajustado
                    if conf >= threshold:
                        # Añadir a detecciones actuales
                        object_id = None

                        # Verificar si corresponde a un objeto ya detectado anteriormente
                        for old_id, old_info in last_detections.items():
                            old_center_x, old_center_y = old_info['center']
                            old_cls = old_info['class']
                            old_area = old_info['area']

                            # Calcular distancia entre centros
                            distance = np.sqrt(
                                (center_x - old_center_x)**2 + (center_y - old_center_y)**2)

                            # Distancia máxima permitida según tamaño
                            max_distance = 100
                            if area < size_categories['small']:
                                max_distance = 50

                            # Diferencia de área permitida
                            area_ratio = max(area, old_area) / \
                                (min(area, old_area) + 1e-5)

                            # Si está cerca y tamaño similar, asignar mismo ID
                            if distance < max_distance and area_ratio < 2.0:
                                object_id = old_id
                                
                                # REGLA SUPER ESTRICTA: Si cambia de una clase a Alpine, rechazar cambio
                                if old_cls != 'Alpine' and cls_name == 'Alpine':
                                    cls_name = old_cls
                                break

                        # Si no se encontró correspondencia, asignar nuevo ID
                        if object_id is None:
                            object_id = id_counter
                            id_counter += 1
                            track_history[object_id] = []
                            class_history[object_id] = []

                        # Actualizar historial de tracking
                        if object_id in track_history:
                            track_history[object_id].append(
                                (center_x, center_y))
                            # Limitar historial a 15 posiciones
                            if len(track_history[object_id]) > 15:
                                track_history[object_id].pop(0)

                        # Actualizar historial de clases
                        if object_id in class_history:
                            class_history[object_id].append(cls_name)
                            # Limitar historial a 10 clases
                            if len(class_history[object_id]) > 10:
                                class_history[object_id].pop(0)

                            # Estabilizar clase usando historial
                            if len(class_history[object_id]) >= 3:
                                # Contar ocurrencias de cada clase
                                class_counts = Counter(class_history[object_id])
                                
                                # Encontrar la clase más común
                                most_common_class = class_counts.most_common(1)[0][0]
                                
                                # REGLA ESTRICTA: Si Alpine no es dominante, usar la más común
                                if cls_name == 'Alpine' and most_common_class != 'Alpine':
                                    cls_name = most_common_class

                        # Almacenar información actual
                        current_detections[object_id] = {
                            'box': (x1, y1, x2, y2),
                            'conf': conf,
                            'class': cls_name,
                            'center': (center_x, center_y),
                            'area': area
                        }

                        # Dibujar bounding box
                        color = team_colors.get(cls_name, (0, 255, 0))
                        cv2.rectangle(original_frame, (x1, y1),
                                      (x2, y2), color, 2)

                        # Etiquetar con clase y confianza
                        label = f"{cls_name}: {conf:.2f}"
                        t_size = cv2.getTextSize(
                            label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                        cv2.rectangle(
                            original_frame, (x1, y1-t_size[1]-3), (x1+t_size[0], y1), color, -1)
                        cv2.putText(original_frame, label, (x1, y1-3),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Actualizar detecciones previas
        last_detections = current_detections

        # Mostrar información de rendimiento
        fps_text = f"FPS: {current_fps:.1f}"
        obj_count = f"Objetos: {len(current_detections)}"
        model_name = "Anti-Alpine Simple"
        cv2.putText(original_frame, fps_text, (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(original_frame, obj_count, (20, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(original_frame, model_name, (20, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Guardar frame procesado si se solicitó
        if output_path:
            out.write(original_frame)

        # Mostrar frame si se solicitó
        if show:
            cv2.imshow('F1 Detection', original_frame)

            # Salir si se presiona 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Liberar recursos
    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()


# Ejemplo de uso
if __name__ == "__main__":
    # Ruta a tu video
    video_path = "../f1-strategy/data/videos/spain_2023_race.mp4.f399.mp4"
    output_path = "../f1-strategy/data/videos/detection_anti_alpine_simple.mp4"

    # Procesar video
    process_video(video_path, output_path, show=True)

KeyError: 'Ferarri'